请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 

## 1.下载paddleX、跑通一个图像分类的示例

### 1.1下载paddle X
根据[飞桨官网](https://www.paddlepaddle.org.cn/paddlex/download)上的安装步骤下载paddleX
<center><img src='https://ai-studio-static-online.cdn.bcebos.com/c72f5190df7e4367a2177002560076cc3a9f3faaf15246c4b79c49d304e77191' width='700'></center>


### 1.2跑通image classification示例
本次试用的示例为**果蔬分类**

<center><img src='https://ai-studio-static-online.cdn.bcebos.com/ac42cb4860b44496ad05ce3d3b37d661c4224843b756487dbb0a755373c468cd' width='700'></center>

## 2.分割任务实现

### 2.1数据集下载和解压
* 安装paddlex

In [2]:
!pip install paddlex

Looking in indexes: https://mirror.baidu.com/pypi/simple/
     |████████████████████████████████| 522kB 10.6MB/s eta 0:00:01
     |████████████████████████████████| 102kB 34.2MB/s ta 0:00:01
     |████████████████████████████████| 1.0MB 18.6MB/s eta 0:00:01
     |████████████████████████████████| 153kB 31.7MB/s eta 0:00:01
  Created wheel for pycocotools: filename=pycocotools-2.0.2-cp37-cp37m-linux_x86_64.whl size=278364 sha256=d98873195355c4cfcf43118d9a2031be50145118907d53e8a338d7972a7f78c9
  Stored in directory: /home/aistudio/.cache/pip/wheels/fb/44/67/8baa69040569b1edbd7776ec6f82c387663e724908aaa60963
Successfully built pycocotools


* 下载和解压数据

In [ ]:
import paddlex as pdx
optic_dataset = 'https://bj.bcebos.com/paddlex/datasets/optic_disc_seg.tar.gz'
pdx.utils.download_and_decompress(optic_dataset, path='./')

### 2.2 数据集划分

In [3]:
import paddlex
!paddlex --split_dataset --format SEG --dataset_dir optic_disc_seg --val_value 0.2 --test_value 0.1

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/setuptools/depends.py:2: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
Dataset Split Done.
Train samples: 267
Eval samples: 76
Test samples: 38
Split files saved in optic_disc_seg


### 2.3 语义分割模型训练及保存

In [10]:
import paddlex as pdx
from paddlex.seg import transforms
# 定义训练和验证时的transforms

train_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(), transforms.ResizeRangeScaling(),
    transforms.RandomPaddingCrop(crop_size=512), transforms.Normalize()
])

eval_transforms = transforms.Compose([
    transforms.ResizeByLong(long_size=512),
    transforms.Padding(target_size=512), transforms.Normalize()
])

# 定义训练和验证所用的数据集

train_dataset = pdx.datasets.SegDataset(
    data_dir='optic_disc_seg',
    file_list='optic_disc_seg/train_list.txt',
    label_list='optic_disc_seg/labels.txt',
    transforms=train_transforms,
    shuffle=True)

eval_dataset = pdx.datasets.SegDataset(
    data_dir='optic_disc_seg',
    file_list='optic_disc_seg/val_list.txt',
    label_list='optic_disc_seg/labels.txt',
    transforms=eval_transforms)

# 初始化模型，并进行训练

num_classes = len(train_dataset.labels)

model = pdx.seg.DeepLabv3p(
    num_classes=num_classes,
    backbone='MobileNetV3_large_x1_0_ssld',
    pooling_crop_size=(512, 512))


model.train(
    num_epochs=10,
    train_dataset=train_dataset,
    train_batch_size=4,
    eval_dataset=eval_dataset,
    learning_rate=0.01,
    save_dir='output/deeplabv3p_mobilenetv3_large_ssld',
    use_vdl=True)

2021-04-18 21:50:19 [INFO]	267 samples in file optic_disc_seg/train_list.txt
2021-04-18 21:50:19 [INFO]	76 samples in file optic_disc_seg/val_list.txt


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/math_op_patch.py:298: UserWarning: /opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddlex/cv/nets/mobilenet_v3.py:231
The behavior of expression A * B has been unified with elementwise_mul(X, Y, axis=-1) from Paddle 2.0. If your code works well in the older versions but crashes in this version, try to use elementwise_mul(X, Y, axis=0) instead of A * B. This transitional warning will be dropped in the future.
  op_type, op_type, EXPRESSION_MAP[method_name]))
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/math_op_patch.py:298: UserWarning: /opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddlex/cv/nets/segmentation/deeplabv3p.py:287
The behavior of expression A * B has been unified with elementwise_mul(X, Y, axis=-1) from Paddle 2.0. If your code works well in the older versions but crashes in this version, try to use elemen

2021-04-18 21:50:22 [INFO]	Decompressing output/deeplabv3p_mobilenetv3_large_ssld/pretrain/MobileNetV3_large_x1_0_ssld_pretrained.tar...
2021-04-18 21:50:22 [INFO]	Load pretrain weights from output/deeplabv3p_mobilenetv3_large_ssld/pretrain/MobileNetV3_large_x1_0_ssld_pretrained.
2021-04-18 21:50:22 [WARNING]	[SKIP] Shape of pretrained weight output/deeplabv3p_mobilenetv3_large_ssld/pretrain/MobileNetV3_large_x1_0_ssld_pretrained/conv14_expand_weights doesn't match.(Pretrained: (672, 112, 1, 1), Actual: (336, 112, 1, 1))
2021-04-18 21:50:22 [WARNING]	[SKIP] Shape of pretrained weight output/deeplabv3p_mobilenetv3_large_ssld/pretrain/MobileNetV3_large_x1_0_ssld_pretrained/conv14_expand_bn_scale doesn't match.(Pretrained: (672,), Actual: (336,))
2021-04-18 21:50:22 [WARNING]	[SKIP] Shape of pretrained weight output/deeplabv3p_mobilenetv3_large_ssld/pretrain/MobileNetV3_large_x1_0_ssld_pretrained/conv14_expand_bn_offset doesn't match.(Pretrained: (672,), Actual: (336,))
2021-04-18 21:50:2

### 2.4 模型预测

In [9]:
#从测试数据中随机挑选一张图
test_jpg = 'optic_disc_seg/JPEGImages/P0099.jpg'
#下载训练好的模型
model = pdx.load_model('output/deeplabv3p_mobilenetv3_large_ssld/epoch_10')
result = model.predict(test_jpg)
pdx.seg.visualize(test_jpg, result, weight=0.0, save_dir='./')

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/math_op_patch.py:298: UserWarning: /opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddlex/cv/nets/mobilenet_v3.py:231
The behavior of expression A * B has been unified with elementwise_mul(X, Y, axis=-1) from Paddle 2.0. If your code works well in the older versions but crashes in this version, try to use elementwise_mul(X, Y, axis=0) instead of A * B. This transitional warning will be dropped in the future.
  op_type, op_type, EXPRESSION_MAP[method_name]))
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/math_op_patch.py:298: UserWarning: /opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddlex/cv/nets/segmentation/deeplabv3p.py:287
The behavior of expression A * B has been unified with elementwise_mul(X, Y, axis=-1) from Paddle 2.0. If your code works well in the older versions but crashes in this version, try to use elemen

2021-04-20 21:24:50 [INFO]	Model[DeepLabv3p] loaded.
2021-04-20 21:24:51 [INFO]	The visualized result is saved as ./visualize_P0099.jpg


### 2.5 模型导出
导出模型保存在`./inference_model路径下`

In [12]:
!paddlex --export_inference --model_dir='output/deeplabv3p_mobilenetv3_large_ssld/epoch_10' --save_dir='./inference_model'

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/setuptools/depends.py:2: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/math_op_patch.py:298: UserWarning: /opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddlex/cv/nets/mobilenet_v3.py:231
The behavior of expression A * B has been unified with elementwise_mul(X, Y, axis=-1) from Paddle 2.0. If your code works well in the older versions but crashes in this version, try to use elementwise_mul(X, Y, axis=0) instead of A * B. This transitional warning will be dropped in the future.
  op_type, op_type, EXPRESSION_MAP[method_name]))
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/math_op_patch.py:298: UserWarning: /opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddlex/cv/n